<font color='tomato'><font color="#CC3D3D"><p>
# Baseline Code (v1.0)

- **문제**   
*"각 패널에게 어떤 온라인 설문조사를 요청해야 응답할까?"*    
**KML Challenge 2023F**에서는 이 문제를 가장 잘 해결할 수 있는 예측 모델을 개발하는 것이 목적
- **평가척도**   
정확도(Accuracy)
- **데이터**   
   - train.csv(학습데이터), test.csv(평가테이터)
   - 각 데이터 포인트는 패널 정보, 설문정보, 응답여부(STATUS)로 구성

### Imports

In [22]:
import pandas as pd
import numpy as np
import os
import random
import pickle
import gc
import re
import warnings; warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

#Base & visualization
import pandas as pd
import random
import os
import numpy as np
import warnings
import matplotlib.pylab as plt
import seaborn as sns

#sklearn module & utils
from tqdm.notebook import tqdm
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import StratifiedKFold , KFold, train_test_split, cross_val_score, cross_validate
from sklearn.feature_selection import SelectPercentile
warnings.filterwarnings('ignore')

#Scaling
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import PowerTransformer

# Encoding
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

#Modeling
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, VotingClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
import time
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import shap
# from featurizer import KMeansFeaturizer

#Scoring
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

from tqdm.auto import trange
import pickle
import gzip
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

2.15.0


In [23]:
# reset_seeds()함수를 아래와 같이 수정해야 함.
def reset_seeds(SEED, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(SEED)
    random.seed(SEED)
    tf.compat.v1.set_random_seed(SEED)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET: ", SEED)  # optional

### Data Loading

In [24]:
train = pd.read_csv('/content/train.csv', encoding='cp949')
test = pd.read_csv('/content/test.csv', encoding='cp949')

In [25]:
train

,userID,surveyID,BIRTH,GENDER,REGION,TYPE,SQ1,SQ2,SQ3,SQ4,...,DQ4,DQ5,DQ6,DQ7,TITLE,IR,LOI,CATEGORIES,CPI,STATUS
0,p00000,s04673,1976.0,1.0,2.0,D,1.0,1976.0,1.0,"1,",...,NaN,NaN,NaN,NaN,해외-일반인의견조사(G-925574),100,13.0,NaN,715.0,1.0
1,p00000,s04675,1976.0,1.0,2.0,D,1.0,1976.0,1.0,"1,",...,NaN,NaN,NaN,NaN,해외-일반인의견조사(G-926698),100,10.0,NaN,550.0,1.0
2,p00000,s04683,1976.0,1.0,2.0,D,1.0,1976.0,1.0,"1,",...,NaN,NaN,NaN,NaN,해외---일반인-의견-조사-(DR-1379333)S,100,12.0,NaN,480.0,1.0
3,p00000,s04684,1976.0,1.0,2.0,D,1.0,1976.0,1.0,"1,",...,NaN,NaN,NaN,NaN,해외---일반인-의견-조사-(DR-8609533)S,100,12.0,NaN,480.0,1.0
4,p00000,s04686,1976.0,1.0,2.0,D,1.0,1976.0,1.0,"1,",...,NaN,NaN,NaN,NaN,해외---일반인-의견-조사-(DR-9958375)L,100,15.0,NaN,600.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789099,p15567,s05436,1979.0,2.0,128.0,C,2.0,1979.0,4.0,1,...,NaN,NaN,NaN,NaN,해외-일반인의견조사(G-975121),100,4.0,NaN,220.0,1.0
789100,p15567,s05440,1979.0,2.0,128.0,C,2.0,1979.0,4.0,1,...,NaN,NaN,NaN,NaN,해외---일반인-의견-조사-(DR-6715815)L,100,15.0,NaN,600.0,0.0
789101,p15567,s05505,1979.0,2.0,128.0,C,2.0,1979.0,4.0,1,...,NaN,NaN,NaN,NaN,해외-일반인의견조사(G-976021),100,5.0,NaN,275.0,0.0
789102,p15567,s05519,1979.0,2.0,128.0,C,2.0,1979.0,4.0,1,...,NaN,NaN,NaN,NaN,해외---일반인-의견-조사-(DN-967191484610),50,10.0,etc,550.0,0.0


In [26]:
train["STATUS"].value_counts()

0.0    483065
1.0    306038
Name: STATUS, dtype: int64

### 데이터 설명
- **패널(Panel) 데이터**
  - userID: 패널 ID
  - BIRTH: 생일(연도)
  - GENDER: 1(남자) 2(여자)
  - REGION: 거주지역
  - TYPE: 패널 구분
  - SQ1 ~ DQ7: 패널 가입 시 설문한 데이터로 각 필드에 대해서는 [상세설명](https://docs.google.com/spreadsheets/d/1pY6MYTSkyHrGLmMTU8tX_xlD8fPut6Ob/edit?usp=sharing&ouid=107121563618260277687&rtpof=true&sd=true) 참조
- **설문(Survey) 데이터**
  - surveyID: 설문 ID
  - TITLE: 설문 제목
  - IR: 난이도(응답가능성, 100이 최고)
  - LOI: 설문하는데 걸리는 시간 (분단위)
  - CATEGORIES: 설문 분류
  - CPI: 패널 응답 리워드 포인트(1P~)
- **응답(Response) 데이터**
  - STATUS: 응답여부 (Target)
  - (test 데이터의) ID: submission을 식별하기 위한 ID로 사용

### Data Preprocessing

In [27]:
# 결측값 비율이 30% 이하인 column만 사용
features = []
for f in train.columns:
    if train[f].isnull().sum()/train.shape[0] <= 0.3:
        print(f, '\t', train[f].nunique(), '\t', train[f].isnull().sum()/train.shape[0])
        features.append(f)

userID 	 14674 	 0.0
surveyID 	 1293 	 0.0
BIRTH 	 76 	 0.0
GENDER 	 2 	 0.0
REGION 	 17 	 0.0
TYPE 	 3 	 0.0
SQ1 	 2 	 0.0008693404164723535
SQ2 	 82 	 0.001215302418946045
SQ3 	 47 	 0.08397498935501531
SQ4 	 52 	 0.0843450292990531
SQ5 	 273 	 0.13528381556803665
SQ6 	 30 	 0.13565132099190982
SQ7 	 1234 	 0.16993957703927492
SQ8 	 1754 	 0.17243607940144773
A1 	 2703 	 0.1963872949573187
B1 	 131 	 0.20272866440925405
B2 	 120 	 0.23041449542772563
B3 	 109 	 0.23580541981792008
B4 	 698 	 0.24689774731847766
B5 	 668 	 0.2542643301770109
TITLE 	 1286 	 0.0
IR 	 43 	 0.0
LOI 	 26 	 1.267260082321215e-06
CPI 	 43 	 1.267260082321215e-06
STATUS 	 2 	 1.267260082321215e-06


In [28]:
# 불필요한 ID들은 제외. SQ1 ~ SQ3는 GENDER, BIRTH, REGION과 중복이어서 제외!
features.remove('userID')
features.remove('surveyID')
features.remove('STATUS')
features.remove('SQ1')
features.remove('SQ2')
features.remove('SQ3')

In [29]:
# 거주 주택 형태: 1/2/3/4/5/6/97이 정상값
train.SQ4.value_counts()

1,                           217950
1                            151820
3                             97234
1                             46019
2                             41761
3,                            32476
4                             31956
4,                            26798
2,                            24451
5                             11207
5,                            10287
2                              7586
4                              5083
3                              4751
5                              3417
6,                             2941
6                              1724
6                               902
97,원룸                           413
97,단독주택                         395
7                               329
97,맑음                           301
97                              237
97,기숙사                          224
7                               213
97,폭염                           179
97,흐림                           165
97                          

In [30]:
train['SQ4'] = train['SQ4'].astype(str)
train['SQ4'] = train['SQ4'].str.strip()

test['SQ4'] = test['SQ4'].astype(str)
test['SQ4'] = test['SQ4'].str.strip()

In [31]:
def handle_profile(x):
    if x in ['1', '2', '3', '4', '5', '6']:
        return int(x)
    elif x in ['1,', '2,', '3,', '4,', '5,', '6,']:
        return int(x[0])
    elif x in ['97,원룸', '97,단독주택', '97,맑음', '97,기숙사', '97,폭염', '97,흐림', '97,덥다', '97,기와집', '97,관사', '97,모름', '97,사택(아파트)', '97,타운하우스', '97,다가구', '97,교회사택', '97,쪽방촌', '97,모텔', '97,잘 모름', '97,맑다', '97,프라이드1', '97,무더움', '97,비', '97,상가 내', '97,투룸', '97,추워요', '97,점포임대주택', '97,사택(공동주거)']:
        return 97
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 99

In [32]:
train.SQ4.value_counts()

1,                           217950
1                            197839
3                            101985
nan                           66557
2                             49347
4                             37039
3,                            32476
4,                            26798
2,                            24451
5                             14624
5,                            10287
6,                             2941
6                              2626
7                               542
97,원룸                           413
97                              402
97,단독주택                         395
97,맑음                           301
97,기숙사                          224
97,폭염                           179
97,흐림                           165
97,덥다                           146
97,기와집                          112
97,관사                            97
97,모름                            89
97,사택(아파트)                       89
97,타운하우스                         85
97,다가구                      

In [33]:
# 최종학력: 1/2/3/4 정상값
train.SQ5.value_counts()[:20]

3.0                 252358
1.0                 115211
3                    82671
4.0                  44836
1                    44135
2.0                  42847
2                    28922
3.0                  20543
4                    11984
99.0                  7310
1.0                   4391
4.0                   3075
2.0                   1567
8.0                   1037
16.0                   644
1,2,3,5                493
12,2,9                 491
1,13,2,3,4,5,8,9       350
1,13,18,2,4,5,9        331
7.0                    314
Name: SQ5, dtype: int64

In [34]:
train['SQ5'] = train['SQ5'].astype(str)
train['SQ5'] = train['SQ5'].str.strip()

test['SQ5'] = test['SQ5'].astype(str)
test['SQ5'] = test['SQ5'].str.strip()

In [35]:
def handle_profile(x):
    if x in ['1', '2', '3', '4']:
        return int(x)
    elif x in ['1.0', '2.0', '3.0', '4.0']:
        return int(x[0])
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 99

train.SQ5 = train.SQ5.apply(handle_profile)
test.SQ5 = test.SQ5.apply(handle_profile)

In [36]:
train.SQ5.value_counts()

3     355572
1     163737
99    117456
2      73336
4      59895
98     19108
Name: SQ5, dtype: int64

In [37]:
# 결혼여부: 1/2/97 정상값
train.SQ6.value_counts()

2                                         182498
1                                         149352
2,                                        107554
1,                                         63628
4                                          58570
3                                          44859
2                                          26894
1                                          18678
5                                          17673
97                                          3897
3                                           2513
4                                           2069
97                                           991
97,이혼                                        986
2,21,5,8,9                                   394
5                                            387
99                                           362
97,사별                                        270
99                                           237
97,별거                                         72
10                  

In [38]:
train['SQ6'] = train['SQ6'].astype(str)
train['SQ6'] = train['SQ6'].str.strip()

test['SQ6'] = test['SQ6'].astype(str)
test['SQ6'] = test['SQ6'].str.strip()

In [39]:
def handle_profile(x):
    if x in ['1', '2']:
        return int(x)
    elif x in ['1,', '2,']:
        return int(x[0])
    elif x in ['97', '97,이혼', '97,사별', '97,별거']:
        return 97
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 99

train.SQ6 = train.SQ6.apply(handle_profile)
test.SQ6 = test.SQ6.apply(handle_profile)

In [40]:
train.SQ6.value_counts()

2     316946
99    233885
1     231658
97      6216
98       399
Name: SQ6, dtype: int64

In [41]:
# 자녀 정보: 1~16/99 정상값
train.SQ7.value_counts()

99.0                 72132
99                   70575
4.0                  33942
16.0                 29861
3.0                  28280
                     ...  
1,13                     5
13,18,2,3,4,5,7          5
1,10,11,2,3,4,5,7        3
1,13,2,5,7,8             3
1,16,2,3,4,5,8,9         2
Name: SQ7, Length: 1234, dtype: int64

In [42]:
train['SQ7'] = train['SQ7'].astype(str)
train['SQ7'] = train['SQ7'].str.strip()

test['SQ7'] = test['SQ7'].astype(str)
test['SQ7'] = test['SQ7'].str.strip()

In [43]:
value1 = pd.Series(range(1,17)).apply(lambda x: str(x)).tolist()
value2 = pd.Series(np.arange(1.0,17.0)).apply(lambda x: str(x)).tolist()
def handle_profile(x):
    if x in value1:
        return int(x)
    elif x in value2:
        return int(x[0])
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 99

train.SQ7 = train.SQ7.apply(handle_profile)
test.SQ7 = test.SQ7.apply(handle_profile)

In [44]:
train.SQ7.value_counts()

99    302479
98    142955
1      92693
3      49368
4      45646
8      45128
2      23994
5      20235
16     17815
7      10109
6       9596
12      5514
14      5477
9       4881
13      4305
15      3925
11      2906
10      2078
Name: SQ7, dtype: int64

In [45]:
# 가족수: 1/2/3/4/5 정상값
train.SQ8.value_counts()[:10]

3.0    116179
4.0    115327
1.0     84230
2.0     52017
1       44829
3       43659
4       38689
5.0     26534
2       14704
99      12504
Name: SQ8, dtype: int64

In [46]:
train['SQ8'] = train['SQ8'].astype(str)
train['SQ8'] = train['SQ8'].str.strip()

test['SQ8'] = test['SQ8'].astype(str)
test['SQ8'] = test['SQ8'].str.strip()

In [47]:
def handle_profile(x):
    if x in ['1', '2', '3', '4', '5']:
        return int(x)
    elif x in ['1.0', '2.0', '3.0', '4.0','5.0']:
        return int(x[0])
    else:
        return 99

train.SQ8 = train.SQ8.apply(handle_profile)
test.SQ8 = test.SQ8.apply(handle_profile)

In [48]:
train.SQ8.value_counts()

99    242410
3     159838
4     154016
1     129059
2      66721
5      37060
Name: SQ8, dtype: int64

In [49]:
# 최근 구입/렌트 가전제품: 1~22/99 정상값
train.A1.value_counts()

1                                      137329
3                                       76869
99                                      59231
2                                       20159
4                                       19431
                                        ...  
1,11,13,15,16,19,2,21,3,4,5,6,7,8,9         1
1,13,2,21,3,4,5,6,9                         1
1,15,2,4,5                                  1
1,11,13,16,2,21,4,5,9                       1
1,13,16,2,21,3,4,5,7,9                      1
Name: A1, Length: 2703, dtype: int64

In [50]:
train['A1'] = train['A1'].astype(str)
train['A1'] = train['A1'].str.strip()

test['A1'] = test['A1'].astype(str)
test['A1'] = test['A1'].str.strip()

In [51]:
#A1_count

In [52]:
str(train['A1'][0])=='nan'

True

In [53]:
def count_numbers(pattern):
    # 문자열에서 ','를 기준으로 분리
    pattern = str(pattern)
    numbers = pattern.split(',')

    # 숫자 개수를 저장할 딕셔너리 초기화
    count_num = 0

    # 각 숫자의 출현 횟수 세기
    for number in numbers:
        number = number.strip()
        if number == 'nan':
            return 0
        else:
            count_num += 1

    return count_num

# train 데이터에 대해 새로운 피처 생성
train['A1_count'] = train['A1'].apply(count_numbers)

# test 데이터에 대해 새로운 피처 생성
test['A1_count'] = test['A1'].apply(count_numbers)

In [54]:
values = pd.Series(range(1,23)).apply(lambda x: str(x)).tolist()
def handle_profile(x):
    if x in values:
        return int(x)
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 99

train.A1 = train.A1.apply(handle_profile)
test.A1 = test.A1.apply(handle_profile)

In [55]:
train.A1.value_counts()

99    257567
98    224483
1     137329
3      76869
2      20159
4      19431
5       9915
13      7176
10      5400
20      5395
21      4435
7       3751
6       2712
9       2399
17      2005
11      1663
12      1499
16      1463
15      1394
8       1305
18      1247
22       799
19       486
14       222
Name: A1, dtype: int64

In [56]:
# 휴대폰 통신망 종류: 1~8/99 정상값
train.B1.value_counts()

4.0                   142547
8.0                    76876
99                     70132
1                      59936
4                      46301
                       ...  
1,10,2,5,6,9               1
10,2                       1
1,10,2,3,4,6,7,8,9         1
10,2,4,6                   1
1,4,6,7                    1
Name: B1, Length: 131, dtype: int64

In [57]:
train['B1'] = train['B1'].astype(str)
train['B1'] = train['B1'].str.strip()

test['B1'] = test['B1'].astype(str)
test['B1'] = test['B1'].str.strip()

In [58]:
def handle_profile(x):
    if x in ['1','1.0']:
        return 1
    elif x in ['2', '2.0']:
        return 2
    elif x in ['3', '3.0']:
        return 3
    elif x in ['4', '5', '6', '7', '4.0', '5.0', '6.0', '7.0']:
        return 4
    elif x in ['8', '8.0']:
        return 5
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 99

train.B1 = train.B1.apply(handle_profile)
test.B1 = test.B1.apply(handle_profile)

In [59]:
# 가입한 이동통신사: 1~6/97 정상값
train.B2.value_counts()

1.0            124643
1               89164
3.0             61118
99              50352
2               41755
                ...  
2,3,4,6,7,9         1
1,3,5,6             1
1,2,5,97            1
6,97                1
1,4,5,97            1
Name: B2, Length: 120, dtype: int64

In [60]:
train['B2'] = train['B2'].astype(str)
train['B2'] = train['B2'].str.strip()

test['B2'] = test['B2'].astype(str)
test['B2'] = test['B2'].str.strip()

In [61]:
def handle_profile(x):
    if x in ['1', '1.0']:
        return 1
    elif x in ['2', '2.0']:
        return 2
    elif x in ['3', '3.0']:
        return 3
    elif x in ['4', '4.0']:
        return 4
    elif x in ['5', '5.0']:
        return 5
    elif x in ['6', '6.0']:
        return 6
    elif re.search(r',.*?,', str(x)):
        return 96
    elif x in ['97', '97.0']:
        return 97
    else:
        return 99

train['B6'] = train['B2'].apply(handle_profile)
test['B6'] = test['B2'].apply(handle_profile)

In [62]:
train['B6'].value_counts()

99    279054
1     213807
3     101212
2      65936
4      56784
5      47389
96     18607
6       4288
97      2027
Name: B6, dtype: int64

In [63]:
def handle_profile(x):
    if x in ['1', '2', '3', '4', '5', '6']:
        return int(x)
    else:
        return 99

train.B2 = train.B2.apply(handle_profile)
test.B2 = test.B2.apply(handle_profile)

In [64]:
train.B2.value_counts()

99    562475
1      89164
2      41755
3      40094
5      29492
4      23829
6       2295
Name: B2, dtype: int64

In [65]:
# 휴대폰 브랜드: 1~7/97/99
train.B3.value_counts()

1                  283617
2                   95652
99                  77351
3                   67291
1,                  23087
                    ...  
1,2,3,4,6,7,8,9         1
1,2,5,6,97              1
1,2,3,4,7               1
1,2,4,7,8,9             1
1,2,3,4,6,7,9           1
Name: B3, Length: 109, dtype: int64

In [66]:
train['B3'] = train['B3'].astype(str)
train['B3'] = train['B3'].str.strip()

test['B3'] = test['B3'].astype(str)
test['B3'] = test['B3'].str.strip()

In [67]:
def handle_profile(x):
    if x in ['1', '2', '3']:
        return int(x)
    elif x in ['1,', '2,', '3,']:
        return int(x[0])
    elif x in ['99', '99,']:
        return 99
    elif re.search(r',.*?,', str(x)):
        return 98
    else:
        return 4  # 기타 제조사

train.B3 = train.B3.apply(handle_profile)
test.B3 = test.B3.apply(handle_profile)

In [68]:
train.B3.value_counts()

1     306704
4     226609
2      98882
99     77561
3      75393
98      3955
Name: B3, dtype: int64

#### B5_count

In [72]:
def count_numbers(pattern):
    # 문자열에서 ','를 기준으로 분리
    pattern = str(pattern)
    numbers = pattern.split(',')

    # 숫자 개수를 저장할 딕셔너리 초기화
    count_num = 0

    # 각 숫자의 출현 횟수 세기
    for number in numbers:
        number = number.strip()
        if number == 'nan':
            return 0
        else:
            count_num+=1

    return count_num

train['B5_count'] = train.B5.apply(count_numbers)
test['B5_count'] = test.B5.apply(count_numbers)

In [73]:
# 서베이 제목에서 유용한 정보 추출
train.TITLE.value_counts()

일반인의견조사(Rm-K)            7214
소비자의견조사(HJ-JJ)           4850
소비자의견조사(JM-WR)           4842
소비자의견조사(SM-OT-Pag)       4720
일반인의견조사(M-TK)            4311
                         ... 
해외-일반인의견조사(G-938771)        1
해외-일반인의견조사(G-1003288)       1
해외-일반인의견조사(G-1004081)       1
해외-일반인의견조사(G-997511)        1
해외-일반인의견조사(G-960414)        1
Name: TITLE, Length: 1286, dtype: int64

In [74]:
word_counts = {}
def count_word(x): # 응답한 서베이 제목에서 한글 단어만 분리하고 빈도 계산
    if x['STATUS'] == 1:
        for w in re.sub(r'[^ ㄱ-ㅣ가-힣]', '', x['TITLE']).split():
            word_counts[w] = word_counts.get(w, 0) + 1
def score_word(x): # 빈도의 합으로 제목을 Encoding
    score = 0
    for w in re.sub(r'[^ ㄱ-ㅣ가-힣]', '', x['TITLE']).split():
        score += word_counts.get(w, 0)
    return score

train.apply(count_word, axis=1)
train.TITLE = train.apply(score_word, axis=1)
test.TITLE = test.apply(score_word, axis=1)

In [75]:
# 응답률 feature 생성
res_freq = train.query('STATUS == 1').groupby('userID')['userID'].agg([('FREQ',np.size)]).reset_index()
send_freq = train.groupby('userID')['userID'].agg([('SEND',np.size)]).reset_index()
train = train.merge(res_freq, how='left').merge(send_freq, how='left')
train['RESRATE'] = train['FREQ'] / train['SEND']
del train['FREQ'], train['SEND']
test = pd.merge(test, train.drop_duplicates(subset=['userID'])[['userID', 'RESRATE']], how='left')

features.extend(['RESRATE'])

### Feature Create

**패널 ID별 응답 리워드, 난이도, 시간 관련**

In [76]:
# 패널의 총 리워드 포인트, 평균/최소/최대 리워드 포인트

# 패널 ID 별 획득 누적 포인트
Points_earned = pd.DataFrame(train.query('STATUS==1').groupby('userID')['CPI'].agg([('Points_earned',np.sum)])).reset_index()

# 패널 ID 별 획득 평균 포인트
Points_earned_mean = pd.DataFrame(train.query('STATUS==1').groupby('userID')['CPI'].agg([('Points_earned_mean',np.mean)])).reset_index()

train = train.merge(Points_earned, how='left')
train = train.merge(Points_earned_mean, how='left')

train['Points_earned'].fillna(0, inplace = True)
train['Points_earned_mean'].fillna(0, inplace = True)

test = test.merge(Points_earned, how='left')
test = test.merge(Points_earned_mean, how='left')

test['Points_earned'].fillna(0, inplace = True)
test['Points_earned_mean'].fillna(0, inplace = True)

In [77]:
# 패널이 응한 설문의 평균난이도
IR = train.query('STATUS==1').groupby('userID')['IR'].agg([('AVG_IR', np.mean)]).reset_index()

train = train.merge(IR, how='left')
test = test.merge(IR, how='left')

In [78]:
# 패널 ID 별 획득 평균 시간
LOI = pd.DataFrame(train.query('STATUS==1').groupby('userID')['LOI'].agg([('AVG_LOI', np.mean)]).reset_index())
train = train.merge(LOI, how='left')
test = test.merge(LOI, how='left')

**BIRTH, REGION 별 응답확률**

In [79]:
BIRTH_status_mean = pd.DataFrame(train.groupby('BIRTH')['STATUS'].agg([('BIRTH_status_mean',np.mean)])).reset_index()

REGION_status_mean = pd.DataFrame(train.groupby('REGION')['STATUS'].agg([('REGION_status_mean',np.mean)])).reset_index()

train = train.merge(BIRTH_status_mean, how='left').merge(REGION_status_mean, how='left')
test = test.merge(BIRTH_status_mean, how='left').merge(REGION_status_mean, how='left')

**IR 별 응답확률**

In [80]:
train_IR = ((train.groupby('IR')['STATUS'].sum())/(train.IR.value_counts().sort_index())).reset_index().rename(columns = {0 : 'STATUS_ratio_IR'})
train = train.merge(train_IR,how='left')
test = test.merge(train_IR,how='left')
test.STATUS_ratio_IR=test.STATUS_ratio_IR.fillna(test.STATUS_ratio_IR.mean())

**LOI별 응답확률**

In [81]:
train_LOI = ((train.groupby('LOI')['STATUS'].sum())/(train.LOI.value_counts().sort_index())).reset_index().rename(columns = {0 : 'STATUS_ratio_LOI'})
train = train.merge(train_LOI,how='left')
test = test.merge(train_LOI,how='left')
test.STATUS_ratio_LOI=test.STATUS_ratio_LOI.fillna(test.STATUS_ratio_LOI.mean())

**CPI 별 응답확률**

In [82]:
train_CPI = ((train.groupby('CPI')['STATUS'].sum())/(train.CPI.value_counts().sort_index())).reset_index().rename(columns = {0 : 'STATUS_ratio_CPI'})
train = train.merge(train_CPI,how='left')
test = test.merge(train_CPI,how='left')
test.STATUS_ratio_CPI=test.STATUS_ratio_CPI.fillna(test.STATUS_ratio_CPI.mean())

**응답률 * 서베이 횟수**

In [83]:
survey_count = train.groupby('userID')['userID'].count()#.reset_index()
survey_count.name = 'user'
survey_count.reset_index()

RES_RATE = train.STATUS.groupby(train.userID).sum() / train.STATUS.groupby(train.userID).count()
RES_RATE_survey_sum = RES_RATE*survey_count
RES_RATE_survey_sum.name = 'RES_RATE_survey_sum'

train = train.merge(RES_RATE_survey_sum,on='userID', how = 'left')
test = test.merge(RES_RATE_survey_sum,on='userID', how = 'left')

**TYPE 별 응답확률**

In [84]:
TYPE_mean = pd.DataFrame(train.groupby('TYPE')['STATUS'].agg([('TYPE_mean',np.mean)])).reset_index()
train = train.merge(TYPE_mean, how='left')
test = test.merge(TYPE_mean, how='left')

**유저아이디별 응답횟수**

In [85]:
res_cnt = train[train.STATUS== 1].groupby("userID").surveyID.count()
train["res_cnt"] = train["userID"].map(dict(res_cnt))
test["res_cnt"] = test["userID"].map(dict(res_cnt))

test["res_cnt"] = test["res_cnt"].fillna(0)

**성별응답확률**

In [ ]:
res_freq = train.query('STATUS == 1').groupby('GENDER')['IR'].agg([('g_i','mean')]).reset_index()
train = pd.merge(train, res_freq, how='left')
test = pd.merge(test, res_freq, how='left')
test.FREQ = test.g_i.fillna(test.g_i.mean())

**성별 응답난이도 평균**

In [86]:
GENDER_IR = train.query('STATUS == 1').groupby('GENDER')['IR'].agg([('GENDER_IR',np.mean)]).reset_index()

train = pd.merge(train, GENDER_IR, how='left')
test = pd.merge(test, GENDER_IR, how='left')

**TYPE 별 CPI 평균**

In [87]:
TYPE_CPI_mean = pd.DataFrame(train.groupby('TYPE')['CPI'].agg([('TYPE_CPI_mean',np.mean)])).reset_index()

train = train.merge(TYPE_CPI_mean, how='left')
test = test.merge(TYPE_CPI_mean, how='left')

**생년->나이**

In [88]:
train["AGE"] = 2022- train["BIRTH"]
test["AGE"] = 2022- test["BIRTH"]

#### 응답률 + 응답률 * 서베이 횟수

In [89]:
train['RESRATE_sum_RES_RATE_survey_sum']= train['RESRATE']+train['RES_RATE_survey_sum']
test['RESRATE_sum_RES_RATE_survey_sum']= test['RESRATE']+test['RES_RATE_survey_sum']

#### 성별 * 응답률

In [90]:
train['GENDER_multiply_RESRATE'] = train['GENDER'] * train['RESRATE']
test['GENDER_multiply_RESRATE'] = test['GENDER'] * test['RESRATE']

#### 응답률 - 획득 포인트 평균

In [91]:
train['RESRATE_diff_Points_earned_mean'] = train['RESRATE'] - train['Points_earned_mean']
test['RESRATE_diff_Points_earned_mean'] = test['RESRATE'] - test['Points_earned_mean']

#### 유저아이디별 응답횟수 + 응답률(인데 -로 만들었습니다...)

In [92]:
train['res_cnt_sum_RESRATE'] = train['res_cnt'] - train['RESRATE']
test['res_cnt_sum_RESRATE'] = test['res_cnt'] - test['RESRATE']

#### 응답률 - 획득 포인트 평균

In [93]:
train['REGION_multiply_RESRATE'] = train['REGION'] * train['RESRATE']
test['REGION_multiply_RESRATE'] = test['REGION'] * test['RESRATE']

#### 유저아이디별 응답횟수 + 응답률(인데 -로 만들었습니다...)

In [94]:
train['Points_earned_mean_sum_RESRATE'] = train['Points_earned_mean'] + train['RESRATE']
test['Points_earned_mean_sum_RESRATE'] = test['Points_earned_mean'] + test['RESRATE']

#### 지역  * 응답률

In [95]:
train['res_cnt_multiply_GENDER'] = train['res_cnt'] * train['GENDER']
test['res_cnt_multiply_GENDER'] = test['res_cnt'] * test['GENDER']

#### 획득 포인트 평균  +  응답률

In [96]:
train['res_cnt_sum_RES_RATE_survey_sum'] = train['res_cnt'] + train['RES_RATE_survey_sum']
test['res_cnt_sum_RES_RATE_survey_sum'] = test['res_cnt'] + test['RES_RATE_survey_sum']

#### 유저아이디별 응답횟수 * 성별

In [97]:
train['RES_RATE_survey_sum_ratio_res_cnt'] = train['RES_RATE_survey_sum'] / train['res_cnt'] * 100
test['RES_RATE_survey_sum_ratio_res_cnt'] = test['RES_RATE_survey_sum'] / test['res_cnt'] * 100

#### (유저아이디별 응답횟수) + (응답률 * 서베이 횟수)

In [98]:
train['res_cnt_ratio_RES_RATE_survey_sum'] = train['res_cnt'] / train['RES_RATE_survey_sum'] * 100
test['res_cnt_ratio_RES_RATE_survey_sum'] = test['res_cnt'] / test['RES_RATE_survey_sum'] * 100

#### 설문 당 평균 응답률

In [99]:
survey_count = train.groupby('surveyID')['surveyID'].count()#.reset_index()
survey_count.name = 'survey'
survey_count.reset_index()


survey_p = train.groupby('surveyID')['IR'].agg([('survey_p',np.mean)]).reset_index()

survey_p['survey_p_survey_sum']= survey_p['survey_p']*survey_count.reset_index()['survey']
train = train.merge(survey_p,on='surveyID', how = 'left')
test = test.merge(survey_p,on='surveyID', how = 'left')
test['survey_p'] = test['survey_p'].fillna(0)

#### 설문지역, 설문대상자유형

In [100]:
train['설문지역'] = train['TITLE'].apply(lambda x: '해외' if '해외' in str(x) else '국내')
train['설문대상자유형'] = train['TITLE'].apply(lambda x: '일반인' if '일반인' in str(x) else ('소비자' if '소비자' in str(x) else '기타'))

test['설문지역'] = test['TITLE'].apply(lambda x: '해외' if '해외' in str(x) else '국내')
test['설문대상자유형'] = test['TITLE'].apply(lambda x: '일반인' if '일반인' in str(x) else ('소비자' if '소비자' in str(x) else '기타'))

#### 동일설문발송횟수

In [101]:
train['동일설문발송횟수'] = train.groupby(['userID', 'TITLE']).cumcount() + 1
test['동일설문발송횟수'] = test.groupby(['userID', 'TITLE']).cumcount() + 1

#### 연령대

In [102]:
train['연령대'] = train['AGE'] // 10
test['연령대'] = test['AGE'] // 10

#### 아줌마, 아저씨

In [103]:
def a_j_c(age, gender):
    return np.where((age == 3.0) & (gender == 1), '맞음', '아님')

train['a_j_c'] = a_j_c(train['연령대'], train['GENDER'])
test['a_j_c'] = a_j_c(test['연령대'], test['GENDER'])

In [104]:
def a_j_m(age, gender):
    return np.where((age == 4.0) & (gender == 2), '맞음', '아님')

train['a_j_m'] = a_j_m(train['연령대'], train['GENDER'])
test['a_j_m'] = a_j_m(test['연령대'], test['GENDER'])

#### 연령대별 응답확률

In [105]:
BIRTH_range_status_mean = pd.DataFrame(train.groupby('연령대')['STATUS'].agg([('연령대_status_mean',np.mean)])).reset_index()

train = train.merge(BIRTH_range_status_mean, how='left')
test = test.merge(BIRTH_range_status_mean, how='left')

In [106]:
train[['LOI']]

,LOI
0,13.0
1,10.0
2,12.0
3,12.0
4,15.0
...,...
789099,4.0
789100,15.0
789101,5.0
789102,10.0


**난이도 대비 응답시간**

In [107]:
train['ResponseTimePerDifficulty'] = train['LOI'] / train['IR']
test['ResponseTimePerDifficulty'] = test['LOI'] / test['IR']

In [108]:
# train 데이터 처리
train['ResponseTimePerDifficulty'].replace([np.inf, -np.inf], np.nan, inplace=True)
train['ResponseTimePerDifficulty'].fillna(train['ResponseTimePerDifficulty'].mean(), inplace=True)

# test 데이터 처리 (train 데이터와 동일한 처리)
test['ResponseTimePerDifficulty'].replace([np.inf, -np.inf], np.nan, inplace=True)
test['ResponseTimePerDifficulty'].fillna(train['ResponseTimePerDifficulty'].mean(), inplace=True)

**유저아이디별 응답횟수 * 성별 응답난이도 평균**

In [109]:
train['res_cnt_GENDER_IR'] = train['res_cnt'] * train['GENDER_IR']
test['res_cnt_GENDER_IR'] = test['res_cnt'] * test['GENDER_IR']

**TYPE 별 응답확률 * 응답률**

In [110]:
train['TYPE_mean_RESRATE'] = train['TYPE_mean'] * train['RESRATE']
test['TYPE_mean_RESRATE'] = test['TYPE_mean'] * test['RESRATE']

**난이도*시간**

In [111]:
train['DifficultyTimesDuration'] = train['IR'] * train['LOI']
test['DifficultyTimesDuration'] = test['IR'] * train['LOI']

**점수/시간  점수*시간**

In [112]:
#점수/시간
train['CPI_Per_LOI'] = train['CPI'] / train['LOI']
test['CPI_Per_LOI'] = test['CPI'] / test['LOI']

#점수*시간
train['CPI_Times_LOI'] = train['CPI'] * train['LOI']
test['CPI_Times_LOI'] = test['CPI'] * test['LOI']

#### A1_분류피처 생성

In [113]:
import pandas as pd
import numpy as np
import re

# Train 데이터에 대한 A1_new 피처 생성
def create_A1_new(train):
    values = {
        1: 1,
        2: 2,
        3: 2,
        4: 3,
        5: 4,
        6: 4,
        7: 4,
        8: 5,
        9: 6,
        10: 6,
        11: 6,
        12: 7,
        20: 7,
        13: 8,
        14: 9,
        19: 9,
        15: 10,
        16: 11,
        17: 11,
        18: 12,
        19: 9,
        21: 13,
        22: 14,
        99: 99
    }
    train['A1_new'] = train['A1'].map(values).fillna(99)
    return train

# Train 데이터에 대한 A1_new 피처 생성
train = create_A1_new(train)

# Test 데이터에 대한 A1_new 피처 생성
test = create_A1_new(test)

In [114]:
A1_new_res_mean = pd.DataFrame(train.groupby('A1_new')['RESRATE'].agg([('A1_new_res_mean',np.mean)])).reset_index()
train = train.merge(A1_new_res_mean, how='left')
test = test.merge(A1_new_res_mean, how='left')

In [115]:
#CPI별응답확률*LOI
train['STATUS_ratio_CPI_MUL_LOI']=train['STATUS_ratio_CPI']*train['LOI']
test ['STATUS_ratio_CPI_MUL_LOI']=test['STATUS_ratio_CPI']*test['LOI']

In [116]:
#TYPE별CPI평균*LOI
train['TYPE_CPI_mean_MUL_LOI']=train['TYPE_CPI_mean']*train['LOI']
test ['TYPE_CPI_mean_MUL_LOI']=test['TYPE_CPI_mean']*test['LOI']

In [117]:
#성별응답난이도평균*시간
train['GENDER_IR_MUL_LOI']=train['GENDER_IR']*train['LOI']
test['GENDER_IR_MUL_LOI'] = test['GENDER_IR']*test['LOI']

#### 데이터 재정의

shap과정 제거하고 shap으로 추출된 features 직접 지정

In [120]:
features = ['Points_earned', 'res_cnt', 'RESRATE', 'survey_p', 'TITLE', 'AVG_IR', 'res_cnt_sum_RESRATE', 'surveyID', 'TYPE', 'STATUS_ratio_CPI', 'Points_earned_mean', 'AVG_LOI', 'GENDER_multiply_RESRATE', 'ResponseTimePerDifficulty', 'CPI', 'STATUS_ratio_LOI', 'IR', 'TYPE_mean', 'REGION_multiply_RESRATE', 'RESRATE_diff_Points_earned_mean', 'GENDER', 'Points_earned_mean_sum_RESRATE', 'LOI', 'BIRTH', 'STATUS_ratio_IR', 'userID', 'REGION_status_mean', 'res_cnt_multiply_GENDER', 'AGE', 'REGION', 'B5', 'SQ8', 'BIRTH_status_mean', 'B6', 'survey_p_survey_sum']
#막줄이 마지막 테스트 때 새로 추가

In [121]:
# 데이터 필터링
X_train = train[features]
X_test = test[features]
y_train = train['STATUS']

# submission을 만들 때 사용하기 위해 ID 저정
ID_test = test.ID

# 메모리 낭비를 막기위해 불필요한 데이터 삭제
del train, test
gc.collect()

8

In [122]:
# 범주형 변수와 수치형 변수를 분리
cat_features = ['GENDER', 'REGION', 'TYPE', 'SQ8', 'B5', 'userID', 'surveyID']
num_features = list(set(X_train.columns) - set(cat_features))

### Feature Engineering

##### Impute missing values

In [123]:
# 응답률 geature 결측값을 평균으로 대체
mean_resrate = X_train['RESRATE'].mean()
X_train['RESRATE'] = X_train['RESRATE'].fillna(mean_resrate)
X_test['RESRATE'] = X_test['RESRATE'].fillna(mean_resrate)

#'AVG_IR' 열의 결측치를 중앙값으로 대체
median_AVG_IR = X_train['AVG_IR'].median()
X_train['AVG_IR'].fillna(median_AVG_IR, inplace=True)
X_test['AVG_IR'].fillna(median_AVG_IR, inplace=True)

# 유저아이디별 응답횟수 결측치 처리
X_train["res_cnt"] = X_train["res_cnt"].fillna(0)

In [124]:
# 결측값 처리: 범주형이냐 수치형이냐에 따라 다르게 처리
if len(num_features) > 0:
    imp = SimpleImputer(strategy='mean')
    X_train[num_features] = imp.fit_transform(X_train[num_features])
    X_test[num_features] = imp.transform(X_test[num_features])

if len(cat_features) > 0:
    imp_cat = SimpleImputer(strategy="constant", fill_value="없음")
    X_train[cat_features] = imp_cat.fit_transform(X_train[cat_features])
    X_test[cat_features] = imp_cat.transform(X_test[cat_features])
# if len(cat_features) > 0:
#     imp = SimpleImputer(strategy="most_frequent")
#     X_train[cat_features] = imp.fit_transform(X_train[cat_features])
#     X_test[cat_features] = imp.transform(X_test[cat_features])

In [125]:
X_train[cat_features] = X_train[cat_features].astype(str)
X_test[cat_features] = X_test[cat_features].astype(str)

In [126]:
X_train['GENDER'] = X_train['GENDER'].astype(float).astype(int)
X_train['REGION'] = X_train['REGION'].astype(float).astype(int)
X_test['GENDER'] = X_test['GENDER'].astype(float).astype(int)
X_test['REGION'] = X_test['REGION'].astype(float).astype(int)

##### Transform features (Feature Scaling)

In [127]:
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

##### Encode categorical features

In [128]:
le = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int)
X_train[cat_features] = le.fit_transform(X_train[cat_features])
X_test[cat_features] = le.transform(X_test[cat_features])

['Points_earned', 'res_cnt', 'RESRATE', 'survey_p', 'TITLE', 'AVG_IR',
       'res_cnt_sum_RESRATE', 'surveyID', 'TYPE', 'STATUS_ratio_CPI',
       'Points_earned_mean', 'AVG_LOI', 'GENDER_multiply_RESRATE',
       'ResponseTimePerDifficulty', 'CPI', 'STATUS_ratio_LOI', 'IR',
       'TYPE_mean', 'REGION_multiply_RESRATE',
       'RESRATE_diff_Points_earned_mean', 'GENDER',
       'Points_earned_mean_sum_RESRATE', 'LOI', 'BIRTH', 'STATUS_ratio_IR',
       'userID', 'REGION_status_mean', 'res_cnt_multiply_GENDER', 'AGE',
       'REGION', 'B5', 'g_i', 'SQ8', 'BIRTH_status_mean', 'B6']

### Model building

In [129]:
X_train.columns

Index(['Points_earned', 'res_cnt', 'RESRATE', 'survey_p', 'TITLE', 'AVG_IR',
       'res_cnt_sum_RESRATE', 'surveyID', 'TYPE', 'STATUS_ratio_CPI',
       'Points_earned_mean', 'AVG_LOI', 'GENDER_multiply_RESRATE',
       'ResponseTimePerDifficulty', 'CPI', 'STATUS_ratio_LOI', 'IR',
       'TYPE_mean', 'REGION_multiply_RESRATE',
       'RESRATE_diff_Points_earned_mean', 'GENDER',
       'Points_earned_mean_sum_RESRATE', 'LOI', 'BIRTH', 'STATUS_ratio_IR',
       'userID', 'REGION_status_mean', 'res_cnt_multiply_GENDER', 'AGE',
       'REGION', 'B5', 'SQ8', 'BIRTH_status_mean', 'B6',
       'survey_p_survey_sum'],
      dtype='object')

In [130]:
# 예측값을 저장할 폴더 생성
folder = 'dnn_ensemble_final'
if not os.path.isdir(folder):
    os.mkdir(folder)

로컬 노트북에서 돌아가지 않아 코랩에서 진행

In [131]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [132]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4503533090654945815
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4829760127293472001
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [133]:
loop = trange(19)
for _ in loop:
    SEED = np.random.randint(1, 10000)
    reset_seeds(SEED)

    # Define the NN architecture
    input_layers = []
    encoded_layers = []
    for feature in cat_features: # 범주형 데이터 처리
        # 입력 레이어 정의
        x = keras.Input(shape=(1,), name=feature)
        input_layers.append(x)
        # 임베딩 레이어 정의
        cardinality = X_train[feature].nunique()
        embedding_dim = int(np.sqrt(cardinality))
        x = keras.layers.Embedding(input_dim=cardinality, output_dim=embedding_dim)(x)
        x = keras.layers.Flatten()(x)
        encoded_layers.append(x)

    cat = keras.layers.concatenate(encoded_layers)
    cat = keras.layers.Dense(8, activation='relu')(cat)
    encoded_layers = []
    for feature in num_features: # 수치형 데이터 처리
        x = keras.Input(shape=(1,), name=feature, dtype=float)
        input_layers.append(x)
        encoded_layers.append(x)
    num = keras.layers.concatenate(encoded_layers)
    x = keras.layers.Concatenate()([cat,num])
    x = keras.layers.Dense(32, activation='relu')(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    output = keras.layers.Dense(1, activation='sigmoid')(x)
    # 모델 생성
    model = keras.Model(inputs=input_layers, outputs=output)

    # Choose the optimizer and the cost function
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Nadam(learning_rate=0.001), metrics=['acc', keras.metrics.BinaryAccuracy()])

    # Train the model
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
    hist = model.fit(
        {key: X_train[key] for key in model.input_names},
        y_train,
        validation_split=0.2,
        batch_size=4096,
        epochs=50,
        callbacks=callbacks,
        shuffle=False,
        verbose=1)
    print("validation accuracy = ", hist.history['val_acc'][-1])

    # Make submissions
    submission = pd.DataFrame({
        "ID": ID_test,
        "STATUS": (model.predict({key: X_test[key] for key in model.input_names}).flatten() >= 0.495).astype(int)
    })
    t = pd.Timestamp.now()
    fname = f"{folder}/loop_submission_{t.month:02}{t.day:02}_{SEED:05}.csv"
    submission.to_csv(fname, index=False)
loop.close()

  0%|          | 0/19 [00:00<?, ?it/s]

RANDOM SEEDS RESET:  1232
Epoch 1/50
155/155 [==============================] - 40s 219ms/step - loss: 0.4468 - acc: 0.7778 - binary_accuracy: 0.7778 - val_loss: nan - val_acc: 0.9149 - val_binary_accuracy: 0.9149
Epoch 2/50
155/155 [==============================] - 2s 12ms/step - loss: 0.3404 - acc: 0.8427 - binary_accuracy: 0.8427 - val_loss: nan - val_acc: 0.9172 - val_binary_accuracy: 0.9172
Epoch 3/50
155/155 [==============================] - 2s 13ms/step - loss: 0.3202 - acc: 0.8519 - binary_accuracy: 0.8519 - val_loss: nan - val_acc: 0.9174 - val_binary_accuracy: 0.9174
Epoch 4/50
155/155 [==============================] - 2s 13ms/step - loss: 0.3112 - acc: 0.8549 - binary_accuracy: 0.8549 - val_loss: nan - val_acc: 0.9178 - val_binary_accuracy: 0.9178
Epoch 5/50
155/155 [==============================] - 2s 16ms/step - loss: 0.3035 - acc: 0.8579 - binary_accuracy: 0.8579 - val_loss: nan - val_acc: 0.9175 - val_binary_accuracy: 0.9175
validation accuracy =  0.9174507856369019


In [135]:
nf = 0
for f in os.listdir(folder):
    ext = os.path.splitext(f)[-1]
    if ext == '.csv':
        s = pd.read_csv(folder+"/"+f)
    else:
        continue
    if len(s.columns) !=2:
        continue
    s.rename(columns={'ID': f+'ID'})
    if nf == 0:
        slist = s
    else:
        # 수정: suffixes 인자를 추가하여 열 이름 충돌을 방지
        slist = pd.merge(slist, s, on="ID", suffixes=('', f'_{nf}'))
    nf += 1

if nf >= 2:
    pred = 0
    for j in range(nf): pred = pred + slist.iloc[:,j+1]
    pred = pred / nf
    pred = pred.apply(lambda x: 1 if x >= 0.5 else 0)
    submission = pd.DataFrame({'ID': slist.ID, 'STATUS': pred})
    t = pd.Timestamp.now()
    fname = f"dnn_final_submission_{t.month:02}{t.day:02}_{t.hour:02}{t.minute:02}.csv"
    submission.to_csv(fname, index=False)